In [4]:
import pandas as pd
import numpy as np

In [33]:
n_teams = 12

In [211]:
df = pd.read_csv('data/ADPvsRank_Clean.csv')

In [212]:
df_rb_adp = df.loc[df['Pos']=='RB'].copy(deep=True)
df_rb_adp['Tier1'] = (df_rb_adp['FPtsRank']<=n_teams).astype(int)
df_rb_adp['Tier2'] = ((df_rb_adp['FPtsRank']>n_teams) & (df_rb_adp['FPtsRank']<=n_teams*2)).astype(int)
df_rb_adp['Tier3'] = ((df_rb_adp['FPtsRank']>n_teams*2) & (df_rb_adp['FPtsRank']<=n_teams*3)).astype(int)
df_rb_adp['Tier4'] = (df_rb_adp['FPtsRank']>n_teams*3).astype(int)
df_rb_adp['Total'] = 1

df_rb_adp = df_rb_adp[['ADPPosRank','Tier1','Tier2','Tier3','Tier4','Total']].groupby('ADPPosRank').sum()

df_rb_adp['Tier1'] = df_rb_adp['Tier1']/df_rb_adp['Total']
df_rb_adp['Tier2'] = df_rb_adp['Tier2']/df_rb_adp['Total']
df_rb_adp['Tier3'] = df_rb_adp['Tier3']/df_rb_adp['Total']
df_rb_adp['Tier4'] = df_rb_adp['Tier4']/df_rb_adp['Total']
df_rb_adp[['Tier1','Tier2','Tier3','Tier4']] = df_rb_adp[['Tier1','Tier2','Tier3','Tier4']]\
    .ewm(span=n_teams/2).mean().ewm(span=n_teams/4).mean()
df_rb_adp.head(24)

,Tier1,Tier2,Tier3,Tier4,Total
ADPPosRank,,,,,
1,0.833333,0.166667,0.000000,0.000000,6
2,0.849537,0.101852,0.048611,0.000000,8
3,0.817342,0.102192,0.043769,0.036697,7
4,0.732619,0.127002,0.079343,0.061036,9
5,0.643004,0.156555,0.098040,0.102400,8
6,0.611378,0.181629,0.087603,0.119390,8
7,0.627281,0.196463,0.070181,0.106074,9
8,0.574689,0.209783,0.111105,0.104423,8
9,0.563964,0.239267,0.108627,0.088141,8


In [213]:
df_rb_pts = df.loc[df['Pos']=='RB'].copy(deep=True)
df_rb_pts['Total'] = 1
df_rb_pts = df_rb_pts[['FPtsRank','FPts','Total']].groupby('FPtsRank').sum()
df_rb_pts['FPts'] = df_rb_pts['FPts']/df_rb_pts['Total']

rb_tier1_ev = df_rb_pts.loc[1:n_teams]['FPts'].sum()/n_teams
rb_tier2_ev = df_rb_pts.loc[n_teams+1:n_teams*2]['FPts'].sum()/n_teams
rb_tier3_ev = df_rb_pts.loc[n_teams*2+1:n_teams*3]['FPts'].sum()/n_teams
rb_tier4_ev = df_rb_pts.loc[n_teams*3+1:n_teams*6]['FPts'].sum()/(n_teams*3)

In [216]:
df_rb = df_rb_adp.copy(deep=True)
df_rb['Tier1_EV'] = rb_tier1_ev
df_rb['Tier2_EV'] = rb_tier2_ev
df_rb['Tier3_EV'] = rb_tier3_ev
df_rb['Tier4_EV'] = rb_tier4_ev
df_rb['Pos'] = 'RB'
df_rb['EV'] = (df_rb['Tier1']*df_rb['Tier1_EV']) + (df_rb['Tier2']*df_rb['Tier2_EV']) + \
    (df_rb['Tier3']*df_rb['Tier3_EV']) + (df_rb['Tier4']*df_rb['Tier4_EV'])
df_rb = df_rb[['Pos','EV']]

# make each EV the max of all EVs >= current, so that curve cannot increase
for i in df_rb.index:
    df_rb.at[i,'EV'] = df_rb.loc[i:]['EV'].max()

df_rb['EVAR'] = df_rb['EV'] - df_rb.loc[n_teams*3]['EV']
df_rb = df_rb.reset_index()
df_rb.head(50)

,ADPPosRank,Pos,EV,EVAR
0,1,RB,260.503488,122.252069
1,2,RB,259.170581,120.919162
2,3,RB,250.485299,112.233880
3,4,RB,235.698783,97.447364
4,5,RB,218.841170,80.589751
5,6,RB,218.369953,80.118534
6,7,RB,218.369953,80.118534
7,8,RB,212.119963,73.868544
8,9,RB,212.119963,73.868544
9,10,RB,207.096353,68.844934


In [217]:
df_wr_adp = df.loc[df['Pos']=='WR'].copy(deep=True)
df_wr_adp['ADPPosRank'] = df_wr_adp['ADPPosRank'] - 1
df_wr_adp['Tier1'] = (df_wr_adp['FPtsRank']<=n_teams).astype(int)
df_wr_adp['Tier2'] = ((df_wr_adp['FPtsRank']>n_teams) & (df_wr_adp['FPtsRank']<=n_teams*2)).astype(int)
df_wr_adp['Tier3'] = ((df_wr_adp['FPtsRank']>n_teams*2) & (df_wr_adp['FPtsRank']<=n_teams*3)).astype(int)
df_wr_adp['Tier4'] = (df_wr_adp['FPtsRank']>n_teams*3).astype(int)
df_wr_adp['Total'] = 1

df_wr_adp = df_wr_adp[['ADPPosRank','Tier1','Tier2','Tier3','Tier4','Total']].groupby('ADPPosRank').sum()

df_wr_adp['Tier1'] = df_wr_adp['Tier1']/df_wr_adp['Total']
df_wr_adp['Tier2'] = df_wr_adp['Tier2']/df_wr_adp['Total']
df_wr_adp['Tier3'] = df_wr_adp['Tier3']/df_wr_adp['Total']
df_wr_adp['Tier4'] = df_wr_adp['Tier4']/df_wr_adp['Total']
#df_wr_adp.loc[1,:] = [0.9,0.,0.,0.1,1.]
df_wr_adp = df_wr_adp.sort_index(ascending=True)
df_wr_adp[['Tier1','Tier2','Tier3','Tier4']] = df_wr_adp[['Tier1','Tier2','Tier3','Tier4']]\
    .ewm(span=n_teams/2).mean().ewm(span=n_teams/4).mean()
df_wr_adp.head(24)

,Tier1,Tier2,Tier3,Tier4,Total
ADPPosRank,,,,,
1,0.666667,0.000000,0.000000,0.333333,3
2,0.796296,0.000000,0.000000,0.203704,2
3,0.869011,0.000000,0.000000,0.130989,2
4,0.913847,0.000000,0.000000,0.086153,3
5,0.942595,0.000000,0.000000,0.057405,3
6,0.894441,0.066940,0.000000,0.038619,5
7,0.815608,0.105168,0.026512,0.052712,6
8,0.791321,0.102088,0.057159,0.049432,6
9,0.771286,0.128638,0.059221,0.040855,7


In [218]:
df_wr_pts = df.loc[df['Pos']=='WR'].copy(deep=True)
df_wr_pts['Total'] = 1
df_wr_pts = df_wr_pts[['FPtsRank','FPts','Total']].groupby('FPtsRank').sum()
df_wr_pts['FPts'] = df_wr_pts['FPts']/df_wr_pts['Total']

wr_tier1_ev = df_wr_pts.loc[1:n_teams]['FPts'].sum()/n_teams
wr_tier2_ev = df_wr_pts.loc[n_teams+1:n_teams*2]['FPts'].sum()/n_teams
wr_tier3_ev = df_wr_pts.loc[n_teams*2+1:n_teams*3]['FPts'].sum()/n_teams
wr_tier4_ev = df_wr_pts.loc[n_teams*3+1:n_teams*6]['FPts'].sum()/(n_teams*3)

In [220]:
df_wr = df_wr_adp.copy(deep=True)
df_wr['Tier1_EV'] = wr_tier1_ev
df_wr['Tier2_EV'] = wr_tier2_ev
df_wr['Tier3_EV'] = wr_tier3_ev
df_wr['Tier4_EV'] = wr_tier4_ev
df_wr['Pos'] = 'WR'
df_wr['EV'] = (df_wr['Tier1']*df_wr['Tier1_EV']) + (df_wr['Tier2']*df_wr['Tier2_EV']) + \
    (df_wr['Tier3']*df_wr['Tier3_EV']) + (df_wr['Tier4']*df_wr['Tier4_EV'])
df_wr = df_wr[['Pos','EV']]

# make each EV the max of all EVs >= current, so that curve cannot increase
for i in df_wr.index:
    df_wr.at[i,'EV'] = df_wr.loc[i:]['EV'].max()

df_wr['EVAR'] = df_wr['EV'] - df_wr.loc[n_teams*3]['EV']
df_wr = df_wr.reset_index()
df_wr.head(50)

,ADPPosRank,Pos,EV,EVAR
0,1,WR,283.235310,109.281602
1,2,WR,283.235310,109.281602
2,3,WR,283.235310,109.281602
3,4,WR,283.235310,109.281602
4,5,WR,283.235310,109.281602
5,6,WR,282.409294,108.455586
6,7,WR,271.729332,97.775625
7,8,WR,268.440589,94.486882
8,9,WR,268.240775,94.287067
9,10,WR,268.240775,94.287067


In [221]:
df_te_adp = df.loc[df['Pos']=='TE'].copy(deep=True)
df_te_adp['Tier1'] = (df_te_adp['FPtsRank']<=n_teams/3).astype(int)
df_te_adp['Tier2'] = ((df_te_adp['FPtsRank']>n_teams/3) & (df_te_adp['FPtsRank']<=n_teams*2/3)).astype(int)
df_te_adp['Tier3'] = ((df_te_adp['FPtsRank']>n_teams*2/3) & (df_te_adp['FPtsRank']<=n_teams)).astype(int)
df_te_adp['Tier4'] = (df_te_adp['FPtsRank']>n_teams).astype(int)
df_te_adp['Total'] = 1

df_te_adp = df_te_adp[['ADPPosRank','Tier1','Tier2','Tier3','Tier4','Total']].groupby('ADPPosRank').sum()

df_te_adp['Tier1'] = df_te_adp['Tier1']/df_te_adp['Total']
df_te_adp['Tier2'] = df_te_adp['Tier2']/df_te_adp['Total']
df_te_adp['Tier3'] = df_te_adp['Tier3']/df_te_adp['Total']
df_te_adp['Tier4'] = df_te_adp['Tier4']/df_te_adp['Total']
df_te_adp = df_te_adp.sort_index(ascending=True)
df_te_adp[['Tier1','Tier2','Tier3','Tier4']] = df_te_adp[['Tier1','Tier2','Tier3','Tier4']]\
    .ewm(span=n_teams/4).mean().ewm(span=n_teams/6).mean()
df_te_adp.head(24)

,Tier1,Tier2,Tier3,Tier4,Total
ADPPosRank,,,,,
1,0.625000,0.000000,0.250000,0.125000,8
2,0.479167,0.166667,0.236111,0.118056,9
3,0.451007,0.205128,0.185287,0.158578,9
4,0.464702,0.226667,0.111468,0.197163,8
5,0.459839,0.190100,0.099832,0.250228,9
6,0.303123,0.232737,0.139446,0.324693,9
7,0.287459,0.236228,0.136448,0.339865,9
8,0.263027,0.269307,0.164659,0.303007,9
9,0.245336,0.296192,0.188576,0.269896,9


In [222]:
df_te_pts = df.loc[df['Pos']=='TE'].copy(deep=True)
df_te_pts['Total'] = 1
df_te_pts = df_te_pts[['FPtsRank','FPts','Total']].groupby('FPtsRank').sum()
df_te_pts['FPts'] = df_te_pts['FPts']/df_te_pts['Total']

te_tier1_ev = df_te_pts.loc[1:n_teams/3]['FPts'].sum()/(n_teams/3)
te_tier2_ev = df_te_pts.loc[n_teams/3+1:n_teams*2/3]['FPts'].sum()/(n_teams/3)
te_tier3_ev = df_te_pts.loc[n_teams*2/3+1:n_teams]['FPts'].sum()/(n_teams/3)
te_tier4_ev = df_te_pts.loc[n_teams+1:n_teams*2]['FPts'].sum()/(n_teams)

In [224]:
df_te = df_te_adp.copy(deep=True)
df_te['Tier1_EV'] = te_tier1_ev
df_te['Tier2_EV'] = te_tier2_ev
df_te['Tier3_EV'] = te_tier3_ev
df_te['Tier4_EV'] = te_tier4_ev
df_te['Pos'] = 'TE'
df_te['EV'] = (df_te['Tier1']*df_te['Tier1_EV']) + (df_te['Tier2']*df_te['Tier2_EV']) + \
    (df_te['Tier3']*df_te['Tier3_EV']) + (df_te['Tier4']*df_te['Tier4_EV'])
df_te = df_te[['Pos','EV']]

# make each EV the max of all EVs >= current, so that curve cannot increase
for i in df_te.index:
    df_te.at[i,'EV'] = df_te.loc[i:]['EV'].max()

df_te['EVAR'] = df_te['EV'] - df_te.loc[n_teams]['EV']
df_te = df_te.reset_index()
df_te.head(24)

,ADPPosRank,Pos,EV,EVAR
0,1,TE,212.746652,40.730712
1,2,TE,204.371066,32.355127
2,3,TE,198.654978,26.639038
3,4,TE,197.002523,24.986583
4,5,TE,189.476122,17.460183
5,6,TE,172.015939,0.000000
6,7,TE,172.015939,0.000000
7,8,TE,172.015939,0.000000
8,9,TE,172.015939,0.000000
9,10,TE,172.015939,0.000000


In [225]:
df_qb_adp = df.loc[df['Pos']=='QB'].copy(deep=True)
df_qb_adp['Tier1'] = (df_qb_adp['FPtsRank']<=n_teams/3).astype(int)
df_qb_adp['Tier2'] = ((df_qb_adp['FPtsRank']>n_teams/3) & (df_qb_adp['FPtsRank']<=n_teams*2/3)).astype(int)
df_qb_adp['Tier3'] = ((df_qb_adp['FPtsRank']>n_teams*2/3) & (df_qb_adp['FPtsRank']<=n_teams)).astype(int)
df_qb_adp['Tier4'] = (df_qb_adp['FPtsRank']>n_teams).astype(int)
df_qb_adp['Total'] = 1

df_qb_adp = df_qb_adp[['ADPPosRank','Tier1','Tier2','Tier3','Tier4','Total']].groupby('ADPPosRank').sum()

df_qb_adp['Tier1'] = df_qb_adp['Tier1']/df_qb_adp['Total']
df_qb_adp['Tier2'] = df_qb_adp['Tier2']/df_qb_adp['Total']
df_qb_adp['Tier3'] = df_qb_adp['Tier3']/df_qb_adp['Total']
df_qb_adp['Tier4'] = df_qb_adp['Tier4']/df_qb_adp['Total']
df_qb_adp = df_qb_adp.sort_index(ascending=True)
df_qb_adp[['Tier1','Tier2','Tier3','Tier4']] = df_qb_adp[['Tier1','Tier2','Tier3','Tier4']]\
    .ewm(span=n_teams/4).mean().ewm(span=n_teams/6).mean()
df_qb_adp.head(24)

,Tier1,Tier2,Tier3,Tier4,Total
ADPPosRank,,,,,
1,0.444444,0.000000,0.222222,0.333333,9
2,0.444444,0.166667,0.166667,0.222222,9
3,0.488400,0.161172,0.139194,0.211233,9
4,0.398730,0.222381,0.205238,0.173651,9
5,0.515709,0.193482,0.144627,0.146182,9
6,0.397490,0.236216,0.161106,0.205187,9
7,0.319104,0.238553,0.109672,0.332671,9
8,0.199306,0.233475,0.176001,0.391219,9
9,0.187040,0.265983,0.165356,0.381620,9


In [226]:
df_qb_pts = df.loc[df['Pos']=='QB'].copy(deep=True)
df_qb_pts['Total'] = 1
df_qb_pts = df_qb_pts[['FPtsRank','FPts','Total']].groupby('FPtsRank').sum()
df_qb_pts['FPts'] = df_qb_pts['FPts']/df_qb_pts['Total']

qb_tier1_ev = df_qb_pts.loc[1:n_teams/3]['FPts'].sum()/(n_teams/3)
qb_tier2_ev = df_qb_pts.loc[n_teams/3+1:n_teams*2/3]['FPts'].sum()/(n_teams/3)
qb_tier3_ev = df_qb_pts.loc[n_teams*2/3+1:n_teams]['FPts'].sum()/(n_teams/3)
qb_tier4_ev = df_qb_pts.loc[n_teams+1:n_teams*2]['FPts'].sum()/(n_teams)

In [227]:
df_qb = df_qb_adp.copy(deep=True)
df_qb['Tier1_EV'] = qb_tier1_ev
df_qb['Tier2_EV'] = qb_tier2_ev
df_qb['Tier3_EV'] = qb_tier3_ev
df_qb['Tier4_EV'] = qb_tier4_ev
df_qb['Pos'] = 'QB'
df_qb['EV'] = (df_qb['Tier1']*df_qb['Tier1_EV']) + (df_qb['Tier2']*df_qb['Tier2_EV']) + \
    (df_qb['Tier3']*df_qb['Tier3_EV']) + (df_qb['Tier4']*df_qb['Tier4_EV'])
df_qb = df_qb[['Pos','EV']]

# make each EV the max of all EVs >= current, so that curve cannot increase
for i in df_qb.index:
    df_qb.at[i,'EV'] = df_qb.loc[i:]['EV'].max()

df_qb['EVAR'] = df_qb['EV'] - df_qb.loc[n_teams]['EV']
df_qb = df_qb.reset_index()
df_qb.head(24)

,ADPPosRank,Pos,EV,EVAR
0,1,QB,287.880564,34.648868
1,2,QB,287.880564,34.648868
2,3,QB,287.880564,34.648868
3,4,QB,287.880564,34.648868
4,5,QB,287.880564,34.648868
5,6,QB,273.744046,20.512349
6,7,QB,255.420560,2.188863
7,8,QB,253.231696,0.000000
8,9,QB,253.231696,0.000000
9,10,QB,253.231696,0.000000


In [228]:
df_out = pd.concat([df_rb,df_wr,df_te,df_qb], ignore_index=False)
df_out.to_csv('data/values.csv', index=False)

# Draft Simulations

In [301]:
sims = 10000
rounds = 12
p_qb = 18
p_te = 18
p_rb = 54
p_wr = 54

df_sim = None

for i in range(sims):
    qbs_taken = 0
    tes_taken = 0
    rbs_taken = 0
    wrs_taken = 0
    teams = [[str(j),[],0,0,0,0,0] for j in range(1,n_teams+1)]
    for n in range(rounds):
        for p in range(n_teams):
            cur_team = p if n%2==0 else n_teams-p-1
            r = np.random.rand()
            if teams[cur_team][2]<2 and (
                (teams[cur_team][3]==2 and r<p_qb/(p_qb+p_rb+p_wr))
                or (r<p_qb/(p_qb+p_te+p_rb+p_wr))
                ):
                teams[cur_team][1].append('QB')
                try:
                    teams[cur_team].append(df_qb.loc[qbs_taken]['EVAR'])
                except KeyError:
                    teams[cur_team].append(-100)
                qbs_taken += 1
                teams[cur_team][2] += 1
            elif teams[cur_team][3]<2 and (
                (teams[cur_team][2]==2 and r<p_te/(p_te+p_rb+p_wr))
                or (r>=p_qb/(p_qb+p_te+p_rb+p_wr) and r<(p_qb+p_te)/(p_qb+p_te+p_rb+p_wr))
                ):
                teams[cur_team][1].append('TE')
                try:
                    teams[cur_team].append(df_te.loc[tes_taken]['EVAR'])
                except KeyError:
                    teams[cur_team].append(-100)
                tes_taken += 1
                teams[cur_team][3] += 1
            elif teams[cur_team][4]<6 and (
                (teams[cur_team][2]==2 and teams[cur_team][3]<2 
                  and r>=p_te/(p_te+p_rb+p_wr) and r<(p_rb+p_te)/(p_te+p_rb+p_wr))
                or (teams[cur_team][2]<2 and teams[cur_team][3]==2 
                  and r>=p_qb/(p_te+p_rb+p_wr) and r<(p_rb+p_qb)/(p_te+p_rb+p_wr))
                or (teams[cur_team][2]==2 and teams[cur_team][3]==2 
                  and r<p_rb/(p_rb+p_wr))
                or (teams[cur_team][2]<2 and teams[cur_team][3]<2 
                  and r<(p_qb+p_te+p_rb)/(p_qb+p_te+p_rb+p_wr))
                ):
                teams[cur_team][1].append('RB')
                try:
                    teams[cur_team].append(df_rb.loc[rbs_taken]['EVAR'])
                    if teams[cur_team][6]==0:
                        teams[cur_team][6] += df_rb.loc[rbs_taken]['EVAR']
                except KeyError:
                    teams[cur_team].append(-100)
                rbs_taken += 1
                teams[cur_team][4] += 1
            else:
                teams[cur_team][1].append('WR')
                try:
                    teams[cur_team].append(df_wr.loc[wrs_taken]['EVAR'])
                except KeyError:
                    teams[cur_team].append(-100)
                wrs_taken += 1
                teams[cur_team][5] += 1
                
    colnames = ['Draft Position','Positions Selected','QBs Selected','TEs Selected','RBs Selected',
                'WRs Selected','Best RB']
    for j in range(1,rounds+1):
        colnames.append('Round {} EVAR'.format(str(j)))
    teams = pd.DataFrame(teams, columns=colnames)
    
    if df_sim is None:
        df_sim = teams.copy(deep=True)
    else:
        df_sim = pd.concat([df_sim, teams], ignore_index=True)
                
df_sim = df_sim.loc[(df_sim['QBs Selected']>0) & (df_sim['TEs Selected']>0) & (df_sim['WRs Selected']>3)
                   & (df_sim['RBs Selected']>3)]

In [305]:
df_sim['Total Value'] = df_sim['Round 1 EVAR'] + df_sim['Round 2 EVAR'] + df_sim['Round 3 EVAR'] + \
    df_sim['Round 4 EVAR'] + df_sim['Round 5 EVAR'] + df_sim['Round 6 EVAR'] + df_sim['Round 7 EVAR'] + \
    df_sim['Round 8 EVAR'] + df_sim['Round 9 EVAR'] + df_sim['Round 10 EVAR'] + df_sim['Round 11 EVAR'] + \
    df_sim['Round 12 EVAR']

In [307]:
df_sim.sort_values(by='Total Value', ascending=False).loc[df_sim['Best RB']<90].head(10)

,Draft Position,Positions Selected,QBs Selected,TEs Selected,RBs Selected,WRs Selected,Best RB,Round 1 EVAR,Round 2 EVAR,Round 3 EVAR,Round 4 EVAR,Round 5 EVAR,Round 6 EVAR,Round 7 EVAR,Round 8 EVAR,Round 9 EVAR,Round 10 EVAR,Round 11 EVAR,Round 12 EVAR,Total Value
20344,5,"[WR, WR, RB, WR, RB, WR, WR, RB, RB, RB, QB, TE]",1,1,5,5,73.868544,109.281602,97.775625,73.868544,75.931447,37.398325,50.975483,48.049721,0.000000,-4.098061,-4.098061,0.000000,-18.608685,466.475940
113802,7,"[WR, WR, RB, WR, WR, WR, WR, RB, RB, TE, QB, RB]",1,1,4,6,60.572292,109.281602,108.455586,60.572292,75.931447,58.497320,55.478900,48.049721,-4.098061,-4.331542,0.000000,0.000000,-53.406906,454.430361
103691,12,"[RB, WR, WR, WR, WR, WR, WR, RB, TE, RB, RB, QB]",1,1,4,6,80.118534,80.118534,109.281602,86.498865,75.931447,55.478900,50.975483,0.000000,0.000000,0.000000,-4.098061,-4.098061,0.000000,450.088710
53065,2,"[WR, RB, WR, WR, RB, WR, WR, WR, RB, QB, TE, RB]",1,1,4,6,66.550379,109.281602,66.550379,109.281602,93.181785,0.000000,55.478900,55.478900,18.295331,-4.535191,0.000000,0.000000,-53.406906,449.606403
95927,12,"[WR, WR, WR, WR, WR, RB, RB, TE, RB, QB, RB, RB]",1,1,5,5,0.721351,109.281602,109.281602,94.287067,93.181785,55.478900,0.721351,0.000000,0.000000,0.000000,0.000000,-4.535191,-12.705267,444.991849
54349,2,"[WR, RB, RB, RB, RB, WR, WR, WR, TE, TE, WR, QB]",1,2,4,5,80.118534,109.281602,80.118534,80.118534,49.071194,42.734642,48.049721,48.049721,17.322023,0.000000,0.000000,-29.893961,0.000000,444.852011
42837,10,"[WR, TE, RB, RB, WR, WR, WR, WR, RB, RB, QB, QB]",2,1,4,5,66.550379,109.281602,32.355127,66.550379,60.572292,66.686978,55.478900,48.049721,48.049721,-4.535191,-17.549857,0.000000,-21.179842,443.759830
57094,11,"[WR, WR, WR, RB, WR, WR, RB, WR, RB, RB, QB, TE]",1,1,4,6,68.844934,108.455586,97.775625,66.686978,68.844934,55.478900,50.975483,0.000000,17.322023,0.000000,-4.098061,-21.179842,0.000000,440.261626
25702,11,"[WR, WR, RB, RB, WR, RB, WR, RB, RB, TE, QB, QB]",2,1,5,4,67.010750,109.281602,108.455586,67.010750,66.550379,48.049721,31.650811,12.944215,0.000000,-4.098061,0.000000,0.000000,0.000000,439.845004
39767,12,"[RB, WR, WR, WR, WR, TE, WR, WR, RB, RB, RB, QB]",1,1,4,6,80.118534,80.118534,109.281602,94.486882,94.287067,66.686978,0.000000,17.322023,13.347562,-2.924637,-4.098061,-29.466291,0.000000,439.041662


# Current Season ADP Board

In [1]:
import urllib.request as urllib2
from bs4 import BeautifulSoup

In [2]:
ppr_dict = {
    '/half-ppr' : 0.5,
    '/ppr' : 1.0,
    '' : 0.0
}

def _scrape_adp():
    scrape_cols = ['OverallRank','Pick','Player','Position','Team','Bye','ADP','ADP_SD','High','Low','N','Chart']
    df = pd.DataFrame(columns=scrape_cols+['ppr'])
    #for ppr in ['/half-ppr','/ppr','']:
    for ppr in ['/half-ppr']:
        url = 'https://fantasyfootballcalculator.com/adp{0}'.format(ppr)
        page = urllib2.urlopen(urllib2.Request(url, headers={'User-Agent' : "Magic Browser"}))
        soup = BeautifulSoup(page, 'html.parser')

        data = []
        for tr in soup.find_all('tr'):
            tds = tr.find_all('td')
            if len(tds)>0:
                # all rows with player data start with a number and player name, e.g. "1. Melvin Gordon"
                # use that fact to check if this table row has player data in it
                row = []
                for row_val in tds:
                    row.append(row_val.text.strip())
                data.append(row)

        temp_df = pd.DataFrame(data, columns=scrape_cols)
        temp_df['ppr'] = ppr_dict[ppr]

        df = df.append(temp_df, ignore_index=True)
        
    num_cols = ['OverallRank','Bye','ADP','ADP_SD','High','Low','N']
    for c in num_cols:
        df[c] = pd.to_numeric(df[c])
                
    return df

In [5]:
df_adp_board = _scrape_adp()

In [6]:
df_adp_board['Player'] = df_adp_board['Player'].str.replace('.','')
df_adp_board['Player'] = df_adp_board['Player'].str.replace("'",'')
df_adp_board['Player'] = df_adp_board['Player'].str.strip()

df_adp_board = df_adp_board[['Player','Position','Team','Bye','ADP','ADP_SD']]

In [7]:
# quick fix
df_adp_board['Player'] = df_adp_board['Player'].str.replace('Pat Mahomes', 'Patrick Mahomes')

In [10]:
teams = 12
rounds = 12
board_data = []
df_adp_board = df_adp_board.sort_values(by='ADP', ascending=True)
for i in range(rounds):
    row = df_adp_board['Player'].iloc[i*teams:(i+1)*teams].tolist()
    if i%2==1:
        row.reverse()
    board_data.append(row)
    
draft_board = pd.DataFrame(board_data)
draft_board

,0,1,2,3,4,5,6,7,8,9,10,11
0,Saquon Barkley,Christian McCaffrey,Alvin Kamara,David Johnson,Ezekiel Elliott,DeAndre Hopkins,Davante Adams,LeVeon Bell,Julio Jones,James Conner,Michael Thomas,Todd Gurley
1,Antonio Brown,Damien Williams,Keenan Allen,Kerryon Johnson,Mike Evans,Travis Kelce,Dalvin Cook,Joe Mixon,JuJu Smith-Schuster,Tyreek Hill,Nick Chubb,Odell Beckham Jr
2,Adam Thielen,Melvin Gordon,Leonard Fournette,TY Hilton,Devonta Freeman,Aaron Jones,Patrick Mahomes,Marlon Mack,Amari Cooper,David Montgomery,George Kittle,Zach Ertz
3,Phillip Lindsay,Sony Michel,Kenny Golladay,Chris Godwin,Robert Woods,Brandin Cooks,Mark Ingram,Derrick Henry,Julian Edelman,Chris Carson,Stefon Diggs,Josh Jacobs
4,Cooper Kupp,Tyler Lockett,AJ Green,Tevin Coleman,Tyler Boyd,Deshaun Watson,Calvin Ridley,James White,OJ Howard,DJ Moore,Evan Engram,Mike Williams
5,Andrew Luck,Christian Kirk,Robby Anderson,Baker Mayfield,Kenyan Drake,Hunter Henry,Alshon Jeffery,Miles Sanders,Tarik Cohen,Jarvis Landry,Aaron Rodgers,Lamar Miller
6,Austin Ekeler,Allen Robinson,Rashaad Penny,Latavius Murray,Dante Pettis,Matt Ryan,Will Fuller,Derrius Guice,Vance McDonald,Jared Cook,Eric Ebron,Darrell Henderson
7,Ronald Jones II,Geronimo Allison,Kareem Hunt,Sammy Watkins,Drew Brees,Royce Freeman,Larry Fitzgerald,Kyler Murray,Jordan Howard,Marvin Jones,Carson Wentz,Curtis Samuel
8,Marquez Valdes-Scantling,Cam Newton,David Njoku,Dede Westbrook,Chicago Defense,Jaylen Samuels,Sterling Shepard,LeSean McCoy,Jared Goff,Corey Davis,Carlos Hyde,DeSean Jackson
9,NKeal Harry,Peyton Barber,LA Rams Defense,Emmanuel Sanders,Jameis Winston,Devin Singletary,DK Metcalf,Courtland Sutton,Adrian Peterson,Donte Moncrief,Russell Wilson,Kalen Ballage
